In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import glob
import os
from google.colab.patches import cv2_imshow
from PIL import Image as im
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing import image

In [ ]:
#paths to validation,test,train sets
path_test = "/content/drive/MyDrive/Colab Notebooks/dataset/dataset/test/"
path_train = "/content/drive/MyDrive/Colab Notebooks/dataset/dataset/train/"
path_val = "/content/drive/MyDrive/Colab Notebooks/dataset/dataset/val/"

# creates np arrays for data and labels, as well as a distribution of each extension
def table_creator(path):
  freq, data, labels, gifs = {}, [], [], []
  for root, dirs, files in os.walk(path, topdown=False):
    for name in files:
        front, sep, extension = name.partition(".")
        label, sep, end = name.partition(" ")
        freq.update({extension: 1 if extension not in freq else freq[extension] + 1})
        #image = cv2.imread(os.path.join(root, name))
        image = im.open(os.path.join(root, name))
        image = np.array(image)
        #exclude gif for now
        if extension != 'gif':
          data.append(image)
          labels.append(label)
        #if extension == 'gif':
          #gifs.append(1);
        #else:
          #gifs.append(0);
  data = np.array(data)
  labels = np.array(labels)
  gifs = np.array(gifs)
  return freq, data, labels, gifs

freq_test, test_data, test_labels, test_gifs = table_creator(path_test)
freq_train, train_data, train_labels, train_gifs = table_creator(path_train)
freq_val, val_data, val_labels, val_gifs = table_creator(path_val)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
classes = ['dipper', 'mabel']
batch_size = 15
IMG_SHAPE = 75
epochs = 50

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255,
                               rotation_range=45,
                               zoom_range=.5, 
                               horizontal_flip=True,
                               vertical_flip=True,
                               brightness_range=(.25,.5),
                               width_shift_range=.15,
                               height_shift_range=.15
                               )

train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,directory=path_train, shuffle=True, target_size=(IMG_SHAPE,IMG_SHAPE), class_mode = 'binary')
image_gen = ImageDataGenerator(rescale=1./255
                               )
validation_data_gen = image_gen.flow_from_directory(batch_size=batch_size,directory=path_val,shuffle=True, target_size=(IMG_SHAPE,IMG_SHAPE), class_mode = 'binary')

Found 143 images belonging to 2 classes.
Found 19 images belonging to 2 classes.


In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(IMG_SHAPE,IMG_SHAPE,3)),
                             tf.keras.layers.MaxPool2D(2,2),
                             tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                             tf.keras.layers.MaxPool2D(2,2),
                             tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
                             tf.keras.layers.MaxPool2D(2,2),
                             tf.keras.layers.Flatten(),
                             #tf.keras.layers.Dropout(.2),
                             tf.keras.layers.Dense(128,activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])
#lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-6 * 10**(epoch / 30))
#early_stopping = tf.keras.callbacks.EarlyStopping(patience=10)
#optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)
model.compile(optimizer='adam',loss='BinaryCrossentropy',metrics=['accuracy'])
history = model.fit(train_data_gen,epochs=20,validation_data=validation_data_gen)#callbacks=[early_stopping],callbacks=[lr_schedule]

In [ ]:
path_test = "/content/drive/MyDrive/Colab Notebooks/dataset/dataset/test"
predictions = []
for root, dirs, files in os.walk(path_test, topdown=False):
    for name in files:
      test_image = image.load_img(os.path.join(root,name), target_size=(IMG_SHAPE,IMG_SHAPE))
      test_image = image.img_to_array(test_image) # convert image to array
      test_image = np.expand_dims(test_image, axis = 0) # expand dimensions
      result = model.predict(test_image) # predict the image
      predictions.append("dipper" if result[0][0] == 0 else "mabel")
print(predictions)
print(test_labels)
#for image in test_data:
  #image = np.reshape(image, (IMG_SHAPE, IMG_SHAPE))
  #img = mpimg.imread(image)
  #imgplot = plt.imshow(img)
  #plt.show()
  #image = im.fromarray(image)
  #image.show()
#path_test = "/content/drive/MyDrive/Colab Notebooks/dataset/dataset/test"
#image_gen = ImageDataGenerator(rescale=1./255
#                               )
#test_data_gen = image_gen.flow_from_directory(batch_size=batch_size,directory=path_test,shuffle=True, target_size=(IMG_SHAPE,IMG_SHAPE), class_mode = 'sparse')
#print(test_data_gen)
#predictions = model.predict(test_data_gen)
#import matplotlib.pyplot as plt
#plt.semilogx(history.history["lr"], history.history["loss"])
#plt.axis([1e-6, 1e-3, 0, 20])

['dipper', 'dipper', 'dipper', 'dipper', 'dipper', 'mabel', 'mabel', 'mabel', 'dipper', 'mabel', 'mabel', 'mabel', 'mabel', 'dipper', 'dipper', 'dipper']
['dipper' 'dipper' 'dipper' 'dipper' 'dipper' 'mabel' 'mabel' 'dipper'
 'mabel' 'mabel' 'mabel' 'mabel' 'dipper' 'mabel' 'dipper']
